# Dose Scaling (Absolute Calibration)

This script sets the absolute dose calibration by comparing calculated dose to a measured calibration point.

**What you need:**
- Absolute dose measurement for 10×10 cm field at 10 cm depth
- Known MU value used for the measurement

**What to do:**
1. Set `CALIBRATION_MU` to your calibration monitor units
2. Run the script - first plot shows uncalibrated dose
3. After `dose_engine.calibrate()`, second plot shows calibrated dose
4. Verify the dose at isocenter matches your measurement (~1.0 Gy typical)

**Output:**
- Calibrated dose engine ready for clinical use


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torch

from pydose_rt.data import MachineConfig, Phantom, Beam
from pydose_rt import DoseEngine

In [ ]:
CALIBRATION_MU = 110

In [ ]:
machine_config = MachineConfig(preset="../../src/pydose_rt/data/machine_presets/umea_10MV.json")

phantom_shape = (100, 100, 100)
phantom_resolution = (2.0, 2.0, 2.0)
phantom = Phantom.from_uniform_water(shape=phantom_shape, spacing=phantom_resolution)

device =  torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32

In [ ]:
beam = Beam.create(0.0, 60, 0.0, (100.0, 100.0))
beam.mu = CALIBRATION_MU * beam.mu

phantom=phantom.to(device).to(dtype)
beam = beam.to(device).to(dtype)

dose_engine = DoseEngine(machine_config, dose_grid_spacing=phantom.resolution, kernel_size=101, dose_grid_shape=phantom.density_image.shape, beam_template=beam)

dose = dose_engine.compute_dose(beam, density_image=phantom.density_image)

In [ ]:

slice_idx = 84
print(dose.shape)

fig, ax = plt.subplots()
im = ax.imshow(dose[0, 50, :, :].cpu().detach().numpy(), cmap='jet', vmin=np.min(dose.cpu().detach().numpy()), vmax=np.max(dose.cpu().detach().numpy()), interpolation='nearest')


plt.suptitle(f"Dose 10cm deep: {np.round(dose[0, 50, 50, 50].detach().cpu().numpy(), 3)} Gy.")
# Add a rectangle around [25, 64]
rect = patches.Rectangle((50 - 1, 20 - 1), 1, 1, linewidth=1.5, edgecolor='white', facecolor='none')
ax.add_patch(rect)

ax.axis('off')
plt.colorbar(im, ax=ax)
plt.show()
print(f"Recommended mean avg. {machine_config.mean_photon_energy_MeV / dose[0, 50, 50, 50]}")

In [ ]:
dose_engine.calibrate(CALIBRATION_MU)
dose = dose_engine.compute_dose(beam, density_image=phantom.density_image)

In [ ]:

slice_idx = 84
print(dose.shape)

fig, ax = plt.subplots()
im = ax.imshow(dose[0, 50, :, :].cpu().detach().numpy(), cmap='jet', vmin=np.min(dose.cpu().detach().numpy()), vmax=np.max(dose.cpu().detach().numpy()), interpolation='nearest')


plt.suptitle(f"Dose 10cm deep: {np.round(dose[0, 50, 50, 50].detach().cpu().numpy(), 3)} Gy.")
# Add a rectangle around [25, 64]
rect = patches.Rectangle((50 - 1, 20 - 1), 1, 1, linewidth=1.5, edgecolor='white', facecolor='none')
ax.add_patch(rect)

ax.axis('off')
plt.colorbar(im, ax=ax)
plt.show()
print(f"Recommended mean avg. {machine_config.mean_photon_energy_MeV / dose[0, 50, 50, 50]}")